In [38]:
# import packages
import numpy as np
import pylab as py
import matplotlib.pyplot as plt
import math

# import spisea packages
from spisea import synthetic, evolution, atmospheres, reddening, ifmr
from spisea.imf import imf, multiplicity

# colormap stuff for plot generation
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

# Isochrone parameters
The difference this time is that we are using 6 filters for chi-squared analysis. The interpolation code will most definitely need to be changed so that this can be performed on the synethic CMD outputs created from Nbody6

In [39]:
# define isochrone parameters
AKs = 0
dist = 4500
evo_model = evolution.Baraffe15()
atm_func = atmospheres.get_merged_atmosphere
red_law = reddening.RedLawHosek18b()
filt_list = ['jwst,F162M', 'jwst,F182M', 'jwst,F200W', 'jwst,F356W', 'jwst,F405N']
filters = ['m_jwst_F162M', 'm_jwst_F182M', 'm_jwst_F200W', 'm_jwst_F356W', 'm_jwst_F405N']
iso_dir = 'isochrones/'
metallicity = 0

In [40]:
# define log age array for isochrone grid
level_ages = np.linspace(1, 10, 19)
level_ages = np.multiply(level_ages, 1e6)
print(level_ages)
log_age_arr = np.log10(level_ages)
print(log_age_arr)

[ 1000000.  1500000.  2000000.  2500000.  3000000.  3500000.  4000000.
  4500000.  5000000.  5500000.  6000000.  6500000.  7000000.  7500000.
  8000000.  8500000.  9000000.  9500000. 10000000.]
[6.         6.17609126 6.30103    6.39794001 6.47712125 6.54406804
 6.60205999 6.65321251 6.69897    6.74036269 6.77815125 6.81291336
 6.84509804 6.87506126 6.90308999 6.92941893 6.95424251 6.97772361
 7.        ]


# Generate isochrone grid

In [41]:
instances = np.empty(len(log_age_arr), dtype=object)

for i in range(len(log_age_arr)):
    my_iso = synthetic.IsochronePhot(log_age_arr[i], AKs, dist, metallicity=metallicity,
                            evo_model=evo_model, atm_func=atm_func,
                            red_law=red_law, filters=filt_list,
                                iso_dir=iso_dir)
    instances[i] = my_iso
        
print("isochrone generation done")

Isochrone generation took 1.208440 s.
Making photometry for isochrone: log(t) = 6.00  AKs = 0.00  dist = 4500
     Starting at:  2024-09-12 03:05:02.052640   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F162M = 21.99
Starting filter: jwst,F182M   Elapsed time: 0.60 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F182M = 22.18
Starting filter: jwst,F200W   Elapsed time: 1.19 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F200W = 22.04
Starting filter: jwst,F356W   Elapsed time: 1.78 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F356W = 21.17
Starting filter: jwst,F405N   Elapsed time: 2.39 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  2344 K  m_jwst_F405N = 20.84
      Time taken: 3.03 seconds
Isochrone generation took 1.117568 s.
Making photometry for isochrone: log(t) = 6.18  AKs =

      Time taken: 3.00 seconds
Changing to logg=5.00 for T=  1893 logg=3.97
Isochrone generation took 1.098937 s.
Making photometry for isochrone: log(t) = 6.74  AKs = 0.00  dist = 4500
     Starting at:  2024-09-12 03:05:39.621136   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F162M = 23.66
Starting filter: jwst,F182M   Elapsed time: 0.59 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F182M = 24.05
Starting filter: jwst,F200W   Elapsed time: 1.18 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F200W = 23.79
Starting filter: jwst,F356W   Elapsed time: 1.76 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F356W = 22.54
Starting filter: jwst,F405N   Elapsed time: 2.37 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1893 K  m_jwst_F405N = 22.18
      Time taken: 3.00 seconds
Changing to log

Starting filter: jwst,F356W   Elapsed time: 1.75 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F356W = 23.04
Starting filter: jwst,F405N   Elapsed time: 2.35 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1602 K  m_jwst_F405N = 22.62
      Time taken: 2.98 seconds
Changing to logg=5.00 for T=  1571 logg=4.05
Isochrone generation took 1.259661 s.
Making photometry for isochrone: log(t) = 7.00  AKs = 0.00  dist = 4500
     Starting at:  2024-09-12 03:06:16.536472   Usually takes ~5 minutes
Starting filter: jwst,F162M   Elapsed time: 0.00 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1571 K  m_jwst_F162M = 25.19
Starting filter: jwst,F182M   Elapsed time: 0.58 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1571 K  m_jwst_F182M = 25.03
Starting filter: jwst,F200W   Elapsed time: 1.16 seconds
Starting synthetic photometry
M =   0.010 Msun  T =  1571 K  m_jwst_F200W = 24.71
Starting filter: jwst,F356W   Elapsed time: 1.

# Function for calculating chi-square value based on photometric magnitudes

In [42]:
def chi_square_reverse_model(iso_grid, sample_mags):
    results = []
    
    for i in range(len(iso_grid)):
        for j in range(len(iso_grid[i].points)):
            arr = []
            chi_square = 0
            for k in range(len(sample_mags)):
                mag_obs = sample_mags[k]
                mag_exp = iso_grid[i].points[j][filters[k]]
                chi_square = chi_square + (((mag_obs - mag_exp) ** 2) / mag_exp)

            arr.append(chi_square)
            arr.append(iso_grid[i].points[j]['mass'])
            arr.append(10 ** log_age_arr[i])
            
            results.append(arr)
    
    sorted_results = sorted(results, key=sort_by_chi_square)
    
    report(sorted_results[:10])
    
def sort_by_chi_square(sub_list):
    return sub_list[0]

def report(output_arr):
    for i in range(len(output_arr)):
        print("Chi-square: " + str(output_arr[i][0]))
        print("Mass (Msun): " + str(output_arr[i][1]))
        print("Age (Myr): " + str(output_arr[i][2]))
        print()

# Define S284 sample magnitudes and run function

In [43]:
sample_mags = []

import csv
with open('s284-no-errors.csv', mode ='r')as file:
  csvFile = csv.reader(file)
  for lines in csvFile:
        sample_mags.append(lines)

sample_mags.pop(0)

['\ufeffF162M', 'F182M', 'F200W', 'F356W', 'F405N']

In [44]:
# convert value array to numbers
for i in range(len(sample_mags)):
    for j in range(len(sample_mags[i])):
        sample_mags[i][j] = float(sample_mags[i][j])
print(sample_mags)

[[18.0957, 17.8409, 17.6518, 16.3455, 9.834], [23.4467, 22.4409, 21.9598, 20.1415, 13.552], [17.7037, 17.4619, 17.4048, 17.0675, 10.811], [16.6887, 16.5989, 16.2548, 15.8125, 9.426], [17.7007, 17.4209, 17.2758, 16.0555, 9.797], [19.4097, 19.0389, 18.7708, 18.0965, 11.888], [20.7147, 20.1869, 19.9608, 18.9855, 12.766], [19.4917, 19.2589, 19.1038, 18.5365, 12.19]]


In [45]:
for i in range(len(sample_mags)):
    chi_square_reverse_model(instances, sample_mags[i])
    print("==========================================")

Chi-square: 2.7863012953898325
Mass (Msun): 0.3
Age (Myr): 1000000.0

Chi-square: 2.81665407642367
Mass (Msun): 0.2
Age (Myr): 1000000.0

Chi-square: 2.838161908052417
Mass (Msun): 0.4
Age (Myr): 1500000.0000000005

Chi-square: 2.8398933119164878
Mass (Msun): 0.4
Age (Myr): 1000000.0

Chi-square: 2.8422610405597224
Mass (Msun): 0.17
Age (Myr): 1000000.0

Chi-square: 2.8647211651801965
Mass (Msun): 0.3
Age (Myr): 1500000.0000000005

Chi-square: 2.8654796842461603
Mass (Msun): 0.5
Age (Myr): 2000000.0000000002

Chi-square: 2.8681579793151615
Mass (Msun): 0.5
Age (Myr): 1500000.0000000005

Chi-square: 2.878640042385607
Mass (Msun): 0.4
Age (Myr): 2000000.0000000002

Chi-square: 2.8835652339049407
Mass (Msun): 0.5
Age (Myr): 2499999.9999999995

Chi-square: 2.4860112443082816
Mass (Msun): 0.02
Age (Myr): 1000000.0

Chi-square: 2.515035381484078
Mass (Msun): 0.015
Age (Myr): 1000000.0

Chi-square: 2.5199821102637703
Mass (Msun): 0.02
Age (Myr): 1500000.0000000005

Chi-square: 2.5225195122510